In [1]:
from medtracker import *
from medtracker import database

/Users/ryanneff/suretify/medtracker/medtracker/__init__.py:14: ExtDeprecationWarning: Importing flask.ext.mail is deprecated, use flask_mail instead.
  from flask.ext.mail import Mail
/Users/ryanneff/suretify/medtracker/medtracker/__init__.py:15: ExtDeprecationWarning: Importing flask.ext.login is deprecated, use flask_login instead.
  import flask.ext.login as flask_login


In [2]:
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.dates as mdates

import pandas as pd
import numpy as np

import datetime
import plotly.graph_objects as go

from scipy.optimize import curve_fit

import warnings
warnings.filterwarnings('ignore')

In [3]:
patients = models.Patient.query.all()
devices = models.Device.query.all()

In [4]:
print(len(patients))
print(len(devices))

1
1


In [6]:
sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)

In [3]:
def number_complete(patients,last_days):
    start_date = (datetime.datetime.utcnow() - datetime.timedelta(days=last_days-1)+datetime.timedelta(days=1)).date()
    outcount = 0
    for p in patients:
        if len(p.surveys.filter(models.SurveyResponse.start_time < start_date).all())>0:
            outcount += 1
    return outcount

In [8]:
df = pd.DataFrame(sres.groupby([sres.end_time.dt.floor('d')])["end_time"].count())
df

,end_time
end_time,
2020-06-08,1


In [78]:
def plotlyBarplot(x=None,y=None,hue=None,data=None,ylabel="",xlabel="",title="",
                    width=600,height=400,colors=["rgba"+str(i) for i in cm.get_cmap("Dark2").colors],
                    stacked=False,percent=False,ordered=False):
    yaxis=go.layout.YAxis(
            title=ylabel,
            automargin=True,
            titlefont=dict(size=12),
        )

    xaxis=go.layout.XAxis(
            title=xlabel,
            automargin=True,
            titlefont=dict(size=12),
            type='category'
        )

    layout = go.Layout(
        autosize=True,
        width=width,
        height=height,
        yaxis=yaxis,
        xaxis=xaxis
    )
    titledict= {'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}

    fig = go.Figure(layout=layout)
    ylab = y
    xlab = x
    
    ix = 0
    if hue != None:
        data = data[[type(i)!=float for i in data[hue]]]
        if percent:
            totalcounts = data.groupby(xlab).sum()[ylab]
        for hue,group in data.groupby(hue):
            counts = group.groupby(xlab).sum()[ylab]
            if percent:
                counts = round(counts/totalcounts.loc[counts.index,]*100,1)
            x = [str(i) for i in counts.index]
            y = counts.values
            trace = go.Bar(x=x,y=y,
                           text=y,
                           textposition='auto',
                           orientation='v',
                         marker=dict(color=colors[ix]),
                         name="%s"%(hue))
            ix += 1
            fig.add_trace(trace)
    else:
        counts = data.groupby(xlab).sum()[ylab]
        x = [str(i) for i in counts.index]
        y = counts.values
        trace = go.Bar(x=x,y=y,
                       text=y,
                       textposition='auto',
                       orientation='v',
                     marker=dict(color=colors[ix]),
                     name="%s"%(ylabel))
        ix += 1
        fig.add_trace(trace)
    fig.update_layout(title=titledict)
    if ordered:
        if stacked:
            if percent==False:
                fig.update_layout(barmode='stack',xaxis={'categoryorder':'total ascending'})
            else:
                fig.update_layout(barmode='stack',xaxis={'categoryorder':'category descending'})
        else:
            fig.update_layout(xaxis={'categoryorder':'total ascending'})
    else:
        if stacked:
            fig.update_layout(barmode='stack')
    return fig

In [102]:
sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)
sres["date"] = sres.end_time.dt.floor('d')
sres = sres.groupby(["date","uniq_id"]).first()
sres = sres.reset_index()

In [103]:
from plotly import offline
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go

patients = models.Patient.query.all()
devices = models.Device.query.all()

def model_to_pd(model):
    res = [r.to_dict() for r in model.query.all()]
    return pd.DataFrame(res)

sres = []
for p in patients:
    sr = [s.to_dict() for s in p.surveys.all()]
    sres.extend(sr)

sres = pd.DataFrame(sres)
sres["date"] = sres.end_time.dt.floor('d')
sres = sres.groupby(["date","uniq_id"]).first()
sres = sres.reset_index()

devs = pts = model_to_pd(models.Device)
devs_per_day = pd.DataFrame(devs.groupby([devs.creation_time.dt.floor("d")])["creation_time"].count())
devs_per_day.columns = ["daily_new_devices"]
pts = model_to_pd(models.Patient)
pts_per_day = pd.DataFrame(pts.groupby([pts.creation_time.dt.floor("d")])["creation_time"].count())
pts_per_day.columns = ["daily_registered_students"]
res_per_day = pd.DataFrame(sres.groupby([sres.end_time.dt.floor('d')])["end_time"].count())
res_per_day.columns = ["daily_completed_surveys"]
df = pd.merge(pts_per_day,res_per_day,left_index=True,right_index=True,how="outer")
df = pd.merge(df,devs_per_day,left_index=True,right_index=True,how="outer")
begin_time = datetime.datetime.utcnow().date() - datetime.timedelta(days=14)
df = df.reindex(pd.date_range(begin_time, datetime.datetime.utcnow().date())).fillna(0).astype(int)
df["total_registered_students"] = df["daily_registered_students"].cumsum()
df["total_completed_surveys"] = df["daily_registered_students"].cumsum()
df["total_devices"] = df["daily_new_devices"].cumsum()
df.reset_index(inplace=True)
df = df.sort_values(by="index",ascending=True)
df["index"] = [datetime.datetime.strftime(a,"%D") for a in df["index"]]
#plotlyBarplot(data=df,x="index",y="daily_new_devices",width=400,title="New Devices")
fig1 = plotlyBarplot(data=df,x="index",y="total_devices",width=500,height=300,title="Total Devices")
fig2 = plotlyBarplot(data=df,x="index",y="total_registered_students",width=500,height=300,title="Total Registered Students")
#plotlyBarplot(data=df,x="index",y="daily_completed_surveys",width=400,title="Daily Completed Screenings")
df["daily_uncompleted_surveys"] = df["total_registered_students"] - df["daily_completed_surveys"]
df2 = df.loc[:,["index","daily_uncompleted_surveys","daily_completed_surveys"]]
df2.columns = ["index","Not Completed","Completed"]
df2 = df2.melt(id_vars="index")
fig3 = plotlyBarplot(data=df2,x="index",y="value",hue="variable",width=600,height=300, title="Screening Status",stacked=True)
fig3.show()

In [81]:
from plotly import offline

In [63]:
df.loc[:,["index","total_registered_students","daily_completed_surveys"]].melt(id_vars="index")

,index,variable,value
0,06/01/20,total_registered_students,0
1,06/02/20,total_registered_students,0
2,06/03/20,total_registered_students,0
3,06/04/20,total_registered_students,0
4,06/05/20,total_registered_students,0
5,06/06/20,total_registered_students,0
6,06/07/20,total_registered_students,0
7,06/08/20,total_registered_students,2
8,06/01/20,daily_completed_surveys,0
9,06/02/20,daily_completed_surveys,0


In [64]:
print(len(patients))
print(number_complete(patients,1))

1
1


In [65]:
def destroy_responses():
    for a in models.SurveyResponse.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.QuestionResponse.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.Patient.query.all():
        db.session.delete(a)
    db.session.commit()
    for a in models.Device.query.all():
        db.session.delete(a)
    db.session.commit()

In [77]:
def plotlyCountplot(x=None,y=None,hue=None,data=None,ylabel="",xlabel="",title="",
                    width=600,height=400,colors=["rgba"+str(i) for i in cm.get_cmap("Dark2").colors],
                    stacked=False,percent=False):
    xaxis=go.layout.XAxis(
            title=ylabel,
            automargin=True,
            titlefont=dict(size=12),
        )

    yaxis=go.layout.YAxis(
            title=xlabel,
            automargin=True,
            titlefont=dict(size=12),
            type='category'
        )

    layout = go.Layout(
        autosize=True,
        width=width,
        height=height,
        yaxis=yaxis,
        xaxis=xaxis
    )
    titledict= {'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}

    fig = go.Figure(layout=layout)
    ylab = y
    xlab = x

    ix = 0
    if hue != None:
        data = data[[type(i)!=float for i in data[hue]]]
        if percent:
            data["count"] = 1
            totalcounts = data.groupby(ylab).count()["count"]
        for hue,group in data.groupby(hue):
            group["count"] = 1
            counts = group.groupby(ylab).count()["count"]
            if percent:
                counts = round(counts/totalcounts.loc[counts.index,]*100)
            x = [str(i) for i in counts.index]
            y = counts.values
            trace = go.Bar(x=y,y=x,
                           text=y,
                           textposition='auto',
                           orientation='h',
                         marker=dict(color=colors[ix]),
                         name="%s"%(hue))
            ix += 1
            fig.add_trace(trace)
    else:
        data["count"] = 1
        counts = data.groupby(ylab).count()["count"]
        x = [str(i) for i in counts.index]
        y = counts.values
        trace = go.Bar(x=y,y=x,
                       text=y,
                       textposition='auto',
                       orientation='h',
                     marker=dict(color=colors[ix]),
                     name="%s"%(ylabel))
        ix += 1
        fig.add_trace(trace)
    fig.update_layout(title=titledict)
    if stacked:
        if percent==False:
            fig.update_layout(barmode='stack',yaxis={'categoryorder':'total ascending'})
        else:
            fig.update_layout(barmode='stack',yaxis={'categoryorder':'category descending'})
    else:
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig
